In [1]:
import tensorflow as tf

In [2]:
import pandas as pd
import requests

In [9]:
df = pd.read_csv('https://raw.githubusercontent.com/chefhaider/Fake-News-Detection-Kaggle/main/train.csv')

In [10]:
df.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [14]:
df= df.dropna()

In [30]:
df.head()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


In [15]:
x = df.drop('target',axis  = 1)

In [16]:
y = df.target

In [19]:
import tensorflow as tf

In [43]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers  import Dense

In [27]:
voc_size=5000

In [28]:
messages=x.copy()

In [29]:
messages.reset_index(inplace=True)

In [37]:
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [41]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re


ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  text = re.sub('[^a-zA-Z]', ' ',messages.text[i]).lower().split()
  text = [ ps.stem(word) for word in text if not word in stopwords.words('english')]
  text = ' '.join(text)
  corpus.append(text)

In [44]:
onehot_rep = [one_hot(words,voc_size) for words in corpus]

In [55]:
sent_length = 0
for x in onehot_rep:
  if sent_length<len(x):
    sent_length = len(x)


In [56]:
embed = pad_sequences(onehot_rep,padding = 'pre' ,maxlen = sent_length)

In [58]:
vector_features = 40
model=Sequential()
model.add(Embedding(voc_size,vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 27, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [59]:
import numpy as np
x_final = np.array(embed)
y_final = np.array(y)

In [60]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [62]:

model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
54/54 [==============================] - 5s 50ms/step - loss: 0.6730 - accuracy: 0.5719 - val_loss: 0.6316 - val_accuracy: 0.6398
Epoch 2/10
54/54 [==============================] - 2s 41ms/step - loss: 0.5241 - accuracy: 0.7490 - val_loss: 0.5056 - val_accuracy: 0.7657
Epoch 3/10
54/54 [==============================] - 2s 41ms/step - loss: 0.2949 - accuracy: 0.8829 - val_loss: 0.5711 - val_accuracy: 0.7680
Epoch 4/10
54/54 [==============================] - 2s 41ms/step - loss: 0.1792 - accuracy: 0.9288 - val_loss: 0.6682 - val_accuracy: 0.7496
Epoch 5/10
54/54 [==============================] - 2s 41ms/step - loss: 0.1341 - accuracy: 0.9612 - val_loss: 0.8734 - val_accuracy: 0.7340
Epoch 6/10
54/54 [==============================] - 2s 40ms/step - loss: 0.0731 - accuracy: 0.9798 - val_loss: 1.0945 - val_accuracy: 0.7078
Epoch 7/10
54/54 [==============================] - 2s 41ms/step - loss: 0.0582 - accuracy: 0.9820 - val_loss: 1.1257 - val_accuracy: 0.7179
Epoch 8/10
54